In [18]:
sc.stop()
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession,Row
from pyspark.sql.functions import *
from pyspark.sql.types import *

conf = SparkConf().setAppName("zzzZ")\
                  .setMaster("spark://10.120.26.200:7077")\
                  .set("spark.executor.memory", '6g')\
                  .set('spark.cores.max', '4')

sc = SparkContext(conf=conf)
spark = SparkSession.builder.appName('sql coming~').getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# 讀取 hdfs 資料，將每一列轉成 Row
lines = sc.textFile("hdfs://10.120.26.200/user/spark/spark_sql_101/data/people.txt")
parts = lines.map(lambda l: l.split(","))
people = parts.map(lambda p: Row(name=p[0], age=int(p[1])))
people.collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(age=29, name='Michael'), Row(age=30, name='Andy'), Row(age=19, name='Justin')]

In [24]:
# (i) spark.createDataFrame()
schemaPeople = spark.createDataFrame(people)

schemaPeople.printSchema()
schemaPeople.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)

+---+-------+
|age|   name|
+---+-------+
| 29|Michael|
| 30|   Andy|
| 19| Justin|
+---+-------+

In [25]:
# (ii) .toDF()
schemaPeople = people.toDF()

schemaPeople.printSchema()
schemaPeople.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)

+---+-------+
|age|   name|
+---+-------+
| 29|Michael|
| 30|   Andy|
| 19| Justin|
+---+-------+

In [26]:
schemaPeople.createOrReplaceTempView("people")
teenagers = spark.sql("SELECT name, age FROM people WHERE age >= 13 AND age <= 19")

teenagers.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+---+
|  name|age|
+------+---+
|Justin| 19|
+------+---+

In [27]:
# collect() 回傳 Rows
for teenName in teenagers.collect():
    print(teenName)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(name='Justin', age=19)